In [1]:
#include <iostream>

/*a workaround to solve cling issue*/
#include "../inc/macos_cling_workaround.hpp"
/*set libtorch path, load libs*/
#include "../inc/load_libtorch.hpp"
/*import custom defined macros*/
#include "../inc/custom_def.hpp"
/*import matplotlibcpp*/
#include "../inc/load_matplotlibcpp.hpp"
/*import opencv*/
#include "../inc/load_opencv.hpp"

/*import libtorch header file*/
#include <torch/torch.h>
#include <opencv2/opencv.hpp>

In [2]:
int num_inputs = 784;
int num_outputs = 10;

auto W = torch::normal(0, 0.01, {num_inputs, num_outputs}, c10::nullopt, torch::requires_grad(true));
auto b = torch::zeros({num_outputs}, torch::requires_grad(true));

# softmax回归的从零开始实现

实现softmax
$$\mathrm{softmax}(\mathbf{X})_{ij} = \frac{\exp(\mathbf{X}_{ij})}{\sum_k \exp(\mathbf{X}_{ik})}$$

In [3]:
torch::Tensor softmax(torch::Tensor X)
{
    torch::Tensor X_exp = torch::exp(X);
    auto partition = X_exp.sum(1, true);
    return X_exp / partition;
}

In [4]:
torch::Tensor X = torch::normal(0, 1, {2, 5});
torch::Tensor X_prob = softmax(X);
printT(X_prob);
printT(X_prob.sum(1));

X_prob = 
 0.1398  0.2593  0.3644  0.1745  0.0620
 0.2893  0.0977  0.2180  0.2187  0.1763
[ CPUFloatType{2,5} ]
<<--->>

X_prob.sum(1) = 
 1.0000
 1.0000
[ CPUFloatType{2} ]
<<--->>



实现softmax回归模型

In [5]:
torch::Tensor net(torch::Tensor X)
{   
    return softmax(torch::mm(X.reshape({-1, W.size(0)}), W) + b);
}

In [6]:
//创建一个数据y_hat，其中包含2个样本在3个类别的预测概率， 
//使用y作为y_hat中概率的索引
torch::Tensor y = torch::tensor(at::ArrayRef<int>({0, 2}));
torch::Tensor y_hat = torch::tensor(at::ArrayRef<float>({0.1, 0.3, 0.6, 0.3, 0.2, 0.5}));

y_hat = y_hat.reshape({2,3});
printT(y_hat);
//test
auto t = y_hat.index({torch::tensor(at::ArrayRef<int>({0, 1})), y});
printT(t);

y_hat = 
 0.1000  0.3000  0.6000
 0.3000  0.2000  0.5000
[ CPUFloatType{2,3} ]
<<--->>

t = 
 0.1000
 0.5000
[ CPUFloatType{2} ]
<<--->>



In [7]:
//实现交叉熵损失函数
//in PyTorch, len(tensor)是去取tensor第一维的长度

torch::Tensor cross_entropy(torch::Tensor y_hat, torch::Tensor y)
{
    return -torch::log(y_hat.index({torch::arange(y_hat.size(0)), y}));
}


auto t = cross_entropy(y_hat, y);
printT(t);

t = 
 2.3026
 0.6931
[ CPUFloatType{2} ]
<<--->>



In [8]:
//将预测类别与真实 y 元素进行比较
float accuracy(torch::Tensor y_hat, torch::Tensor y)
{
    if (y_hat.dim() > 1 && y_hat.size(1) >1) {
        y_hat = torch::argmax(y_hat, 1);
    }
    
    torch::Tensor cmp = (y_hat.to(y.dtype()) == y);
    return (cmp.to(y.dtype()).sum()).item<float>();
}

printT(accuracy(y_hat, y) / y.size(0));

accuracy(y_hat, y) / y.size(0) = 
0.5
<<--->>



In [9]:
//Accumulator 实例中创建了 2 个变量，用于分别存储正确预测的数量和预测的总数量
class Accumulator {
public:
    //"""在`n`个变量上累加。"""
    Accumulator(int n) { data = torch::zeros({n}, torch::kFloat);}
    
//     void add(float b) { data += b; }

    void reset(void) { data = torch::zeros({data.size(0)}, torch::kFloat); }
   
    float __getitem__(int idx) { return data[idx].item<float>(); }

private:
    torch::Tensor data;
}